In [1]:
import os

# Define the target directory
current_directory = os.getcwd()  # change your directory here

# Check if the directory exists
if os.path.exists(current_directory):
    # Change the current working directory
    two_folders_up = os.path.dirname(os.path.dirname(current_directory))
    os.chdir(two_folders_up)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {current_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop\azure-ai-agent-services-demo


### **Instantiate the PolicyIndexingPipeline Class**

Settings for running the pipeline are in `src\pipeline\policyIndexer\settings.yaml`.

The PolicyIndexingPipeline automates the process of indexing policy documents into Azure AI Search.

In [2]:
from src.azureAIsearch.run import PolicyIndexingPipeline

indexer = PolicyIndexingPipeline()

### **Upload Document to Landing Zone Blob Storage**

In [3]:
indexer.upload_documents(local_path="data/product_data")

2025-03-19 18:57:32,513 - micro - MainProcess - INFO     Starting document upload from local path: data/product_data (run.py:upload_documents:154)
2025-03-19 18:57:32,515 - micro - MainProcess - INFO     Connected to Blob container: agentic-samples (run.py:upload_documents:158)
2025-03-19 18:57:32,518 - micro - MainProcess - INFO     Scanning directory: data/product_data (run.py:upload_documents:162)
2025-03-19 18:57:32,519 - micro - MainProcess - INFO     Preparing to upload file: data/product_data\Product A Techncial Architecture.pdf to blob path: agentic_ocr\Product A Techncial Architecture.pdf (run.py:upload_documents:170)
2025-03-19 18:57:32,794 - micro - MainProcess - INFO     Successfully uploaded data/product_data\Product A Techncial Architecture.pdf to agentic_ocr\Product A Techncial Architecture.pdf (run.py:upload_documents:176)


### **Create Data Source (Connect Blob)**

In [4]:
indexer.create_data_source()

2025-03-19 18:57:33,128 - micro - MainProcess - INFO     Data source 'ai-agentic-blob' created or updated (run.py:create_data_source:205)


### **Create Index**

In [5]:
indexer.create_index()

2025-03-19 18:57:33,550 - micro - MainProcess - INFO     Index 'ai-agentic-index' created or updated successfully. (run.py:create_index:331)


### **Create Skillset**

In [6]:
indexer.create_skillset()

2025-03-19 18:57:33,702 - micro - MainProcess - INFO     Skillset 'ai-agentic-skillset' created or updated (run.py:create_skillset:549)


### **Create Indexer**

In [7]:
indexer.create_indexer()

2025-03-19 18:57:34,033 - micro - MainProcess - INFO     Indexer 'ai-agentic-indexer' created or updated (run.py:create_indexer:583)


### **Create Run Indexer**

In [8]:
from src.azureAIsearch.run import IndexerRunner

In [9]:
indexer = IndexerRunner(indexer_name="ai-agentic-indexer")
indexer.monitor_indexer_status()

2025-03-19 18:57:34,316 - micro - MainProcess - INFO     Indexer 'ai-agentic-indexer' has been started. (run.py:run_indexer:655)
2025-03-19 18:57:34,374 - micro - MainProcess - INFO     Indexer Status: running (run.py:monitor_indexer_status:700)
2025-03-19 18:57:34,375 - micro - MainProcess - INFO     Last Run Time: 2025-03-19 23:54:43.885000+00:00 (run.py:monitor_indexer_status:701)
2025-03-19 18:57:34,377 - micro - MainProcess - INFO     Execution Status: success (run.py:monitor_indexer_status:702)
2025-03-19 18:57:34,378 - micro - MainProcess - INFO     Indexer 'ai-agentic-indexer' completed successfully. (run.py:monitor_indexer_status:712)


### **Test Search** 

In [10]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential

credential = (
    AzureKeyCredential(os.getenv("AZURE_AI_SEARCH_ADMIN_KEY"))
    if os.getenv("AZURE_AI_SEARCH_ADMIN_KEY")
    else DefaultAzureCredential()
)
index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME", "ai-policies-index")


search_client = SearchClient(
    endpoint=os.environ["AZURE_AI_SEARCH_SERVICE_ENDPOINT"],
    index_name=index_name,
    credential=AzureKeyCredential(os.environ["AZURE_AI_SEARCH_ADMIN_KEY"]),
)

In [11]:
SEARCH_QUERY = "What is the main feature of product A?"

In [12]:
vector_query = VectorizableTextQuery(
    text=SEARCH_QUERY, k_nearest_neighbors=5, fields="vector", weight=0.5
)

search_results = search_client.search(
    vector_query,
    search_mode="all",
    top=5,
    include_total_count=True,
    filter=None,
)

print("Search results:")
for result in search_results:
    print(f"Document ID: {result['id']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Vector: {result['vector']}")
    print()

Search results:
